In [ ]:
import gdal
from gdalconst import *
import cv2
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
from PIL import Image
import scipy.misc
from keras.utils import to_categorical
import os
googleDriveURL = 'https://drive.google.com/drive/folders/1Hnb5jOfElWn-_n-lbNPs_TK8w3VOUu9L?usp=sharing'

In [ ]:
img_filename = 'coastalAreaLandmask.tif'
if os.path.isfile(img_filename):
    # get binary land mask
    dataset = gdal.Open(img_filename, GA_ReadOnly)
    xSize = dataset.RasterXSize
    ySize = dataset.RasterYSize
    bandCount = dataset.RasterCount
    print ('Dataset size is ',xSize,'x',ySize,'x',bandCount)
    band = dataset.GetRasterBand(3)
    bandArray = band.ReadAsArray(0, 0, xSize, ySize).astype(np.float32)
    landmask = np.nan_to_num(bandArray)
else:
    print('Please first download the file coastalAreaLandmask.tif from ', googleDriveURL)
        

img_filename = 'coastalAreaShipmask.tif'
if os.path.isfile(img_filename):
    # get binary ship mask
    dataset = gdal.Open(img_filename, GA_ReadOnly)
    xSize = dataset.RasterXSize
    ySize = dataset.RasterYSize
    bandCount = dataset.RasterCount
    print ('Dataset size is ',xSize,'x',ySize,'x',bandCount)
    band = dataset.GetRasterBand(3)
    bandArray = band.ReadAsArray(0, 0, xSize, ySize).astype(np.float32)
    shipmask = bandArray
    # get binary sea mask
    seamask = 1.0-(landmask+shipmask)
else:
    print('Please first download the file coastalAreaShipmask.tif from ', googleDriveURL)

In [ ]:
categorical_mask = np.dstack((landmask,seamask))
categorical_mask = np.dstack((categorical_mask,shipmask)).astype(np.uint8)

In [ ]:
img_filename = 'coastalAreaSARData.tif'
if os.path.isfile(img_filename):
    # get train data
    dataset = gdal.Open(img_filename, GA_ReadOnly)
    xSize = dataset.RasterXSize
    ySize = dataset.RasterYSize
    bandCount = dataset.RasterCount
    print ('Dataset size is ',xSize,'x',ySize,'x',bandCount)
    band = dataset.GetRasterBand(1)
    bandArray1 = band.ReadAsArray(0, 0, xSize, ySize).astype(np.float32)
    band = dataset.GetRasterBand(2)
    bandArray2 = band.ReadAsArray(0, 0, xSize, ySize).astype(np.float32)
    train_imgs = np.dstack((bandArray1, bandArray2))
else:
    print('Please first download the file coastalAreaSARData.tif from ', googleDriveURL)

In [ ]:
# save images and labels as np arrays
npy_path = '../MulticlassClassificationLandvsSeavsShip/MulticlassClassifier/npydata'
if not os.path.lexists(npy_path):
    os.mkdir(npy_path)
np.save(npy_path + '/imgs.npy', train_imgs)
np.save(npy_path + '/labels.npy', categorical_mask)

npy_path = '../MulticlassClassificationLandvsSeavsShip/OnevsAllClassifiers/npydata'
if not os.path.lexists(npy_path):
    os.mkdir(npy_path)
np.save(npy_path + '/imgs.npy', train_imgs)
np.save(npy_path + '/labels.npy', categorical_mask)